(15p) Using the test dataset, calculate the perplexity of each language model. Report the results obtained. If you experience variable overflow, use probabilities in log space.

In [287]:
import json
import os
import math
from collections import defaultdict

# ---------- Utilidades de carga/normalización ----------

def read_ngram_model(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            model = json.load(f)
        print(f"[OK] Modelo cargado desde '{file_path}'")
        return model
    except FileNotFoundError:
        print(f"[ERROR] Archivo no encontrado: '{file_path}'")
        return None
    except Exception as e:
        print(f"[ERROR] No se pudo cargar el modelo desde '{file_path}': {e}")
        return None

def _normalize_ngram_keys(model_dict, n):
    """
    Convierte llaves 'a b' / 'a b c' en tuplas ('a','b'[, 'c']) para O(1) consistente y evita splits repetidos.
    Devuelve (dict_normalizado, seguidores_por_contexto)
    seguidores_por_contexto:
        - bigrama: dict prev -> #tipos_siguientes
        - trigrama: dict (prev1,prev2) -> #tipos_siguientes
    """
    out = {}
    followers = defaultdict(int)
    if n == 1:
        # unigramas ya están por token -> p(token)
        # no se necesita followers
        for k, v in model_dict.items():
            out[(k,)] = float(v)
        return out, {}
    elif n == 2:
        next_types = defaultdict(set)
        for k, v in model_dict.items():
            a, b = k.split(' ', 1)
            out[(a, b)] = float(v)
            next_types[a].add(b)
        followers = {a: len(bs) for a, bs in next_types.items()}
        return out, followers
    elif n == 3:
        next_types = defaultdict(set)
        for k, v in model_dict.items():
            a, b, c = k.split(' ', 2)
            out[(a, b, c)] = float(v)
            next_types[(a, b)].add(c)
        followers = {ctx: len(cs) for ctx, cs in next_types.items()}
        return out, followers
    else:
        raise ValueError("n debe ser 1, 2 o 3")

# ---------- Perplejidad optimizada ----------

def calculate_perplexity(
    testing_file,
    unigram_model, bigram_model, trigram_model,
    vocab_size=None
):
    """
    Calcula perplejidad usando:
      - Accesos O(1) (tuplas como claves)
      - Conteos de contexto precomputados (evita sum(...) por llave)
      - Log-probabilidades para estabilidad
      - Lectura streaming del archivo de prueba
    """
    if any(m is None for m in (unigram_model, bigram_model, trigram_model)):
        return None, None, None

    # Normaliza llaves y precomputa seguidores/contexts
    uni, _ = _normalize_ngram_keys(unigram_model, 1)
    bi, followers_bi = _normalize_ngram_keys(bigram_model, 2)
    tri, followers_tri = _normalize_ngram_keys(trigram_model, 3)

    # Tamaño de vocabulario: por defecto = #unigramas distintos
    if vocab_size is None:
        vocab_size = len(uni)

    # Precalcula denominadores de Laplace para velocidad
    # Nota: tu fórmula original usa (#tipos_siguientes + V) como denominador de suavizado.
    # Mantengo esa lógica para equivalencia funcional, pero ahora O(1).
    laplace_uni_denom = (len(uni) + vocab_size)

    total_log_u = 0.0
    total_log_b = 0.0
    total_log_t = 0.0
    total_tokens = 0

    # Lectura streaming (sin cargar todo a memoria)
    try:
        with open(testing_file, 'r', encoding='utf-8') as f:
            for raw in f:
                s = raw.strip()
                if not s:
                    continue
                tokens = s.split()
                n = len(tokens)
                if n == 0:
                    continue

                total_tokens += n

                # -------- Unigram --------
                for w in tokens:
                    p = uni.get((w,), 0.0)
                    if p <= 0.0:
                        p = 1.0 / laplace_uni_denom
                    total_log_u += math.log(p)

                # -------- Bigram --------
                # P(w1) + Π P(w_i | w_{i-1})
                # w1 como unigrama
                p1 = uni.get((tokens[0],), 0.0)
                if p1 <= 0.0:
                    p1 = 1.0 / laplace_uni_denom
                total_log_b += math.log(p1)

                for i in range(n - 1):
                    a, b_ = tokens[i], tokens[i + 1]
                    p = bi.get((a, b_), 0.0)
                    if p <= 0.0:
                        context_types = followers_bi.get(a, 0)
                        p = 1.0 / (context_types + vocab_size)
                    total_log_b += math.log(p)

                # -------- Trigram --------
                # P(w1) * P(w2|w1) * Π P(w_i | w_{i-2}, w_{i-1})
                # w1 como unigrama
                p1 = uni.get((tokens[0],), 0.0)
                if p1 <= 0.0:
                    p1 = 1.0 / laplace_uni_denom
                total_log_t += math.log(p1)

                if n >= 2:
                    p2 = bi.get((tokens[0], tokens[1]), 0.0)
                    if p2 <= 0.0:
                        ctx_types = followers_bi.get(tokens[0], 0)
                        p2 = 1.0 / (ctx_types + vocab_size)
                    total_log_t += math.log(p2)

                for i in range(n - 2):
                    a, b_, c = tokens[i], tokens[i + 1], tokens[i + 2]
                    p = tri.get((a, b_, c), 0.0)
                    if p <= 0.0:
                        ctx_types = followers_tri.get((a, b_), 0)
                        p = 1.0 / (ctx_types + vocab_size)
                    total_log_t += math.log(p)

    except Exception as e:
        print(f"[ERROR] No se pudo leer el archivo de prueba '{testing_file}': {e}")
        return None, None, None

    if total_tokens == 0:
        return float('inf'), float('inf'), float('inf')

    ppl_u = math.exp(-total_log_u / total_tokens)
    ppl_b = math.exp(-total_log_b / total_tokens)
    ppl_t = math.exp(-total_log_t / total_tokens)
    return ppl_u, ppl_b, ppl_t

# ---------- Script principal ----------

if __name__ == '__main__':
    group_code = "my_group"

    training_file_20N = os.path.join("tercer-punto", f"20N_{group_code}_training.txt")
    testing_file_20N  = os.path.join("tercer-punto", f"20N_{group_code}_testing.txt")
    training_file_BAC = os.path.join("tercer-punto", f"BAC_{group_code}_training.txt")
    testing_file_BAC  = os.path.join("tercer-punto", f"BAC_{group_code}_testing.txt")

    print("\n\n--- Calculando la perplejidad ---")

    # Modelos 20N
    unigrams_20N = read_ngram_model(os.path.join("cuarto-punto", "models", f"20N_{group_code}_unigrams.json"))
    bigrams_20N  = read_ngram_model(os.path.join("cuarto-punto", "models", f"20N_{group_code}_bigrams.json"))
    trigrams_20N = read_ngram_model(os.path.join("cuarto-punto", "models", f"20N_{group_code}_trigrams.json"))

    # vocab: usa #unigramas; evita re-leer el corpus completo
    vocab_size_20N = len(unigrams_20N) if unigrams_20N else None

    if all([unigrams_20N, bigrams_20N, trigrams_20N]):
        pp_uni_20N, pp_bi_20N, pp_tri_20N = calculate_perplexity(
            testing_file_20N,
            unigrams_20N, bigrams_20N, trigrams_20N,
            vocab_size=vocab_size_20N
        )
        print(f"\nResultados de Perplejidad para 20N (Corpus):")
        print(f"  Unigramas: {pp_uni_20N:.2f}")
        print(f"  Bigramas:  {pp_bi_20N:.2f}")
        print(f"  Trigramas: {pp_tri_20N:.2f}")

    # Modelos BAC
    unigrams_BAC = read_ngram_model(os.path.join("cuarto-punto", "models", f"BAC_{group_code}_unigrams.json"))
    bigrams_BAC  = read_ngram_model(os.path.join("cuarto-punto", "models", f"BAC_{group_code}_bigrams.json"))
    trigrams_BAC = read_ngram_model(os.path.join("cuarto-punto", "models", f"BAC_{group_code}_trigrams.json"))

    vocab_size_BAC = len(unigrams_BAC) if unigrams_BAC else None

    if all([unigrams_BAC, bigrams_BAC, trigrams_BAC]):
        pp_uni_BAC, pp_bi_BAC, pp_tri_BAC = calculate_perplexity(
            testing_file_BAC,
            unigrams_BAC, bigrams_BAC, trigrams_BAC,
            vocab_size=vocab_size_BAC
        )
        print(f"\nResultados de Perplejidad para BAC (Corpus):")
        print(f"  Unigramas: {pp_uni_BAC:.2f}")
        print(f"  Bigramas:  {pp_bi_BAC:.2f}")
        print(f"  Trigramas: {pp_tri_BAC:.2f}")



--- Calculando la perplejidad ---
[OK] Modelo cargado desde 'cuarto-punto/models/20N_my_group_unigrams.json'
[OK] Modelo cargado desde 'cuarto-punto/models/20N_my_group_bigrams.json'
[OK] Modelo cargado desde 'cuarto-punto/models/20N_my_group_trigrams.json'

Resultados de Perplejidad para 20N (Corpus):
  Unigramas: 1256.35
  Bigramas:  3104.00
  Trigramas: 15012.65
[OK] Modelo cargado desde 'cuarto-punto/models/BAC_my_group_unigrams.json'
[OK] Modelo cargado desde 'cuarto-punto/models/BAC_my_group_bigrams.json'
[OK] Modelo cargado desde 'cuarto-punto/models/BAC_my_group_trigrams.json'

Resultados de Perplejidad para BAC (Corpus):
  Unigramas: 825.99
  Bigramas:  1163.46
  Trigramas: 12544.12


(15p) Using your best language model, build a method/function that automatically generates sentences by receiving the first word of a sentence as input. Take different tests and document them.

In [ ]:
import json, math, random
from typing import Dict, List, Tuple, Iterable

# ---------------- Utilidades ----------------

def load_unigram_model(path:str) -> Dict[str, float]:
    """Carga un JSON {token: prob} o {token: count}. Normaliza a probas."""
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)
    # Si ya son probabilidades que suman ~1, úsalo; si parecen conteos, normaliza.
    vals = list(raw.values())
    s = sum(vals)
    # Si hay NaNs o s==0, error
    if not math.isfinite(s) or s <= 0:
        raise ValueError("Modelo vacío o inválido")
    # Normaliza siempre (robusto ante floats que no sumen 1 exactamente)
    return {tok: float(c)/s for tok, c in raw.items()}

def _build_temperature_probs(p: Dict[str,float], temperature: float) -> Tuple[List[str], List[float]]:
    """Aplica temperatura y devuelve listas paralelas (vocab, cdf)."""
    if temperature <= 0:
        raise ValueError("temperature debe ser > 0")
    # p^ (1/T), renormaliza
    powed = {t: (pi ** (1.0/temperature)) for t, pi in p.items() if pi > 0.0}
    z = sum(powed.values())
    vocab = []
    cdf = []
    acc = 0.0
    for t, w in powed.items():
        acc += w / z
        vocab.append(t)
        cdf.append(acc)
    # Asegura que el último sea 1.0 exacto
    cdf[-1] = 1.0
    return vocab, cdf

def _sample_from_cdf(vocab: List[str], cdf: List[float]) -> str:
    r = random.random()
    # búsqueda lineal (rápida en práctica). Cambia a bisect si tu vocab es enorme.
    for t, c in zip(vocab, cdf):
        if r <= c:
            return t
    return vocab[-1]

def _should_stop(token: str, stop_tokens: Iterable[str]) -> bool:
    return token in stop_tokens or any(token.endswith(st) for st in stop_tokens)

# ---------------- Generador ----------------

class UnigramSentenceGenerator:
    def __init__(self, unigram_probs: Dict[str,float], temperature: float = 1.0,
                 stop_tokens: Iterable[str] = (".", "!", "?", "</s>")):
        self.vocab, self.cdf = _build_temperature_probs(unigram_probs, temperature)
        self.stop_tokens = set(stop_tokens)

    def generate(self, first_word: str, max_len: int = 30, seed: int = None) -> str:
        """
        Genera una oración que inicia con first_word y luego muestrea unigramas.
        No re-tokeniza; asume que los tokens del modelo son por palabra.
        """
        if seed is not None:
            random.seed(seed)
        tokens = [first_word]
        # Si el primer token ya es stop, devuelve inmediato
        if _should_stop(first_word, self.stop_tokens):
            return first_word
        # Completa hasta max_len o hasta stop
        for _ in range(max_len - 1):
            nxt = _sample_from_cdf(self.vocab, self.cdf)
            tokens.append(nxt)
            if _should_stop(nxt, self.stop_tokens):
                break
        # Pegado simple (si tu corpus maneja signos como tokens separados, esto es suficiente)
        sent = " ".join(tokens)
        # Arreglo menor de espacios antes de puntuación común
        sent = sent.replace(" .", ".").replace(" ,", ",").replace(" !", "!").replace(" ?", "?")
        return sent

if __name__ == '__main__':
    # 1) Carga tu mejor modelo: BAC_unigramas
    uni = load_unigram_model("cuarto-punto/models/BAC_my_group_unigrams.json")

    # 2) Crea el generador (puedes jugar con la temperatura)
    gen_cool = UnigramSentenceGenerator(uni, temperature=0.8)  # más conservador
    gen_neutral = UnigramSentenceGenerator(uni, temperature=1.0)
    gen_hot = UnigramSentenceGenerator(uni, temperature=1.3)   # más diverso

    # 3) Pruebas
    print(gen_neutral.generate("Colombia", max_len=20, seed=42))
    print(gen_cool.generate("Economía", max_len=20, seed=42))
    print(gen_hot.generate("Gobierno", max_len=20, seed=42))
    print(gen_neutral.generate("Bogotá", max_len=30, seed=7))

Colombia okay <s> and it world where flicks i have <s> it one <s> </s>
Economía do <s> </s>
Gobierno excellent i their one dat wash thunderbolts this hes i at gov i im rape taller your families endangered
Bogotá to just NUM i there the today will <s> was day i youll nasty that it thinking willis some what okinawa <s> brain take can a to nap </s>
